In [33]:
from pandas import read_csv
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from numpy import log
import numpy as np
import pandas as pd
import datetime as datetime
import csv

def zscore(df, col):
    col_mean = df[col].rolling(window=14, center=True).mean()
    col_std = df[col].rolling(window=14, center=True).std()
    return (df[col] - col_mean)/col_std

df = pd.read_csv('./data/11-26/daily_plutchik_threshold_5.csv', parse_dates=['month/day'])
approval_rating_df = pd.read_csv('./data/approval_polllist.csv', parse_dates=["enddate"])
topic_df = pd.read_excel('./data/TopicScoresPerDay_Final.xlsx')
sentiment_df = pd.read_csv('./data/12-6/daily_senti_corrected.csv', parse_dates=['month/day'])

In [34]:
df["Anticipation Z-Score"] = zscore(df, "Anticipation")
df["Trust Z-Score"] = zscore(df, "Trust")
df["Surprise Z-Score"] = zscore(df, "Surprise")
df["Sadness Z-Score"] = zscore(df, "Sadness")
df["Joy Z-Score"] = zscore(df, "Joy")
df["Fear Z-Score"] = zscore(df, "Fear")
df["Disgust Z-Score"] = zscore(df, "Disgust")
df["Anger Z-Score"] = zscore(df, "Anger")

start_date = datetime.datetime(2017, 7, 12, 0, 0)
end_date = datetime.datetime(2017, 12, 31, 0, 0)

dated_emotion_df = df[(df["month/day"] > start_date) & (df["month/day"] < end_date)]

In [35]:
topic_df = topic_df.drop(['Filename', 'Segment', 'WC', 'WPS', 'Sixltr', 'Dic', 
                          'AllPunc', 'Period', 'Comma', 'Colon', 'SemiC', 'QMark', 
                          'Exclam', 'Dash', 'Quote', 'Apostro', 'Parenth', 'OtherP', 
                          'Em1', 'Em2', 'Em3', 'Em4', 'Em5', 'Em6', 'Em7', 'Em8', 'Gallup Value'], axis=1)

for (columnName, columnData) in topic_df.iteritems():
    topic_df[columnName + " Z-Score"] = zscore(topic_df, columnName)
    
topic_df = topic_df[66:237]
topic_df.fillna(0, inplace=True)
topic_df.replace(np.inf, 0, inplace=True)
topic_df.replace(-np.inf, 0, inplace=True)
topic_df.head()

,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,...,topic492 Z-Score,topic493 Z-Score,topic494 Z-Score,topic495 Z-Score,topic496 Z-Score,topic497 Z-Score,topic498 Z-Score,topic499 Z-Score,topic500 Z-Score,topic501 Z-Score
66,6.46,0.0,0.00,0.05,0.0,0.01,0.64,0.57,0.19,0.65,...,0.194023,2.163111,1.089956,-0.491032,1.429682,0.413594,-0.702665,1.065521,1.432840,-0.507075
67,6.12,0.0,0.00,0.04,0.0,0.01,0.46,0.65,0.08,0.49,...,0.769534,-0.151553,-0.433117,-0.479883,0.629919,1.120638,-0.401764,-1.296174,-1.321625,-0.372797
68,6.20,0.0,0.01,0.01,0.0,0.02,0.61,0.51,0.10,0.69,...,0.461597,-1.572928,-0.890207,1.424433,1.097163,1.309574,-0.475466,-1.355222,-2.163202,-0.066748
69,6.08,0.0,0.00,0.03,0.0,0.02,0.47,0.50,0.07,0.67,...,-0.503236,0.630641,2.372378,2.064031,-0.495156,-0.901418,-0.321563,0.105594,0.360875,-0.514077
70,5.97,0.0,0.00,0.06,0.0,0.02,0.45,0.63,0.06,0.21,...,-0.493518,-0.165357,-1.091842,-0.820724,-0.338300,-1.210769,-1.583182,1.308417,0.067327,-0.723097


In [36]:
for (columnName, columnData) in sentiment_df.iteritems():
    if columnName != "month/day":
        sentiment_df[columnName+" Z-Score"] = zscore(sentiment_df, columnName)
sentiment_df = sentiment_df[(sentiment_df["month/day"] > start_date) & (sentiment_df["month/day"] < end_date)]
sentiment_df.head()

,month/day,pos_sentiment,neg_sentiment,neu_sentiment,pos_sentiment Z-Score,neg_sentiment Z-Score,neu_sentiment Z-Score
66,2017-07-13,0.418653,0.391710,0.189637,1.339725,-0.882616,-0.535477
67,2017-07-14,0.389543,0.390280,0.220177,0.723188,-0.960567,0.695292
68,2017-07-15,0.412959,0.376269,0.210773,1.445375,-1.252709,0.201804
69,2017-07-16,0.340331,0.465886,0.193783,-0.453674,0.663924,-0.590644
70,2017-07-17,0.357298,0.470588,0.172113,0.058015,0.762438,-1.515081


In [37]:
def check_stationarity(column, title):
    result = adfuller(column)
    for key, value in result[4].items():
        if result[0] > value:
            print('Found a non-stationary time series:', title)
            print('ADF Statistic: %f' % result[0])
            print('p-value: %f' % result[1])
            for key, value in result[4].items():
                print('\t%s: %.3f' % (key, value))
        if result[1] > 0.05:
            print('Found a p-value greater than threshold')

In [40]:
for (columnName, columnData) in sentiment_df.iteritems():
    if columnName.endswith("Z-Score"):
        check_stationarity(sentiment_df[columnName], columnName)
        
sentiment_df.head()

,month/day,pos_sentiment,neg_sentiment,neu_sentiment,pos_sentiment Z-Score,neg_sentiment Z-Score,neu_sentiment Z-Score
66,2017-07-13,0.418653,0.391710,0.189637,1.339725,-0.882616,-0.535477
67,2017-07-14,0.389543,0.390280,0.220177,0.723188,-0.960567,0.695292
68,2017-07-15,0.412959,0.376269,0.210773,1.445375,-1.252709,0.201804
69,2017-07-16,0.340331,0.465886,0.193783,-0.453674,0.663924,-0.590644
70,2017-07-17,0.357298,0.470588,0.172113,0.058015,0.762438,-1.515081


In [41]:
for i in range(1, 502):
    title = 'topic'+str(i)+' Z-Score'
    check_stationarity(topic_df[title], title)

/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:867: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


In [43]:
for (columnName, columnData) in dated_emotion_df.iteritems():
    if columnName.endswith('Z-Score'):
        check_stationarity(dated_emotion_df[columnName], columnName)

# p-values are all <= 0.05 and ADF Statistics are < values at 1%, 5%, and 10%

In [44]:
truncated_approvals = approval_rating_df.loc[approval_rating_df['pollster'] == 'Ipsos']
truncated_approvals = truncated_approvals.loc[truncated_approvals['subgroup'] == 'All polls']
truncated_approvals = truncated_approvals[(truncated_approvals['enddate'] > start_date) & (truncated_approvals['enddate'] < end_date)]
adjusted_and_unadjusted_approvals = truncated_approvals[['adjusted_approve', 'approve']]
truncated_approvals = truncated_approvals['adjusted_approve']

approvals_array = truncated_approvals.values

In [45]:
to_merge_sentiment = sentiment_df[['pos_sentiment', 'neg_sentiment', 'neu_sentiment', 'pos_sentiment Z-Score', 'neg_sentiment Z-Score', 'neu_sentiment Z-Score']]
to_merge_sentiment = to_merge_sentiment.reset_index(drop=True)
to_merge_emotion = dated_emotion_df.drop('month/day', axis=1)
to_merge_emotion = to_merge_emotion.reset_index(drop=True)
to_merge_approvals = adjusted_and_unadjusted_approvals.reset_index(drop=True)
to_merge_topics = topic_df.reset_index(drop=True)
all_data_to_upload = pd.concat([to_merge_sentiment, to_merge_emotion, to_merge_topics, to_merge_approvals], axis=1)
print(len(to_merge_sentiment), len(dated_emotion_df), len(topic_df), len(truncated_approvals))
all_data_to_upload = all_data_to_upload.rename(columns={'adjusted_approve': 'Ipsos Adjusted Approval', 'approve': 'Ipsos Approval'})
all_data_to_upload.to_csv("12-6_all_data.csv")
all_data_to_upload.head()

171 171 171 171


,pos_sentiment,neg_sentiment,neu_sentiment,pos_sentiment Z-Score,neg_sentiment Z-Score,neu_sentiment Z-Score,Anger,Disgust,Fear,Joy,...,topic494 Z-Score,topic495 Z-Score,topic496 Z-Score,topic497 Z-Score,topic498 Z-Score,topic499 Z-Score,topic500 Z-Score,topic501 Z-Score,Ipsos Adjusted Approval,Ipsos Approval
0,0.418653,0.391710,0.189637,1.339725,-0.882616,-0.535477,0.025688,0.022018,0.036697,0.122936,...,1.089956,-0.491032,1.429682,0.413594,-0.702665,1.065521,1.432840,-0.507075,41.35173,40.6
1,0.389543,0.390280,0.220177,0.723188,-0.960567,0.695292,0.015248,0.017789,0.054638,0.121982,...,-0.433117,-0.479883,0.629919,1.120638,-0.401764,-1.296174,-1.321625,-0.372797,42.25173,41.5
2,0.412959,0.376269,0.210773,1.445375,-1.252709,0.201804,0.013717,0.019204,0.046639,0.130316,...,-0.890207,1.424433,1.097163,1.309574,-0.475466,-1.355222,-2.163202,-0.066748,40.95173,40.2
3,0.340331,0.465886,0.193783,-0.453674,0.663924,-0.590644,0.022744,0.030814,0.038151,0.118855,...,2.372378,2.064031,-0.495156,-0.901418,-0.321563,0.105594,0.360875,-0.514077,40.95173,40.2
4,0.357298,0.470588,0.172113,0.058015,0.762438,-1.515081,0.022989,0.007663,0.034483,0.099617,...,-1.091842,-0.820724,-0.338300,-1.210769,-1.583182,1.308417,0.067327,-0.723097,38.45173,37.7


In [46]:
def find_significant_time_series(time_series, columnName, result_set):
    x = np.asarray([approvals_array, time_series[columnName]]).T
    results = grangercausalitytests(x, maxlag=7, verbose=False)
    for lag, test in results.items():
        for test, values in results[lag][0].items():
            if values[1] <= 0.05:
                print("found a significant test result for column:", columnName, "with", lag, "lags")
                result_set.add(columnName)

In [47]:
significant_emotions = set()
for (columnName, columnData) in dated_emotion_df.iteritems():
    if columnName.endswith('Z-Score'):
        find_significant_time_series(dated_emotion_df, columnName, significant_emotions)

found a significant test result for column: Trust Z-Score with 1 lags
found a significant test result for column: Trust Z-Score with 1 lags
found a significant test result for column: Trust Z-Score with 1 lags
found a significant test result for column: Trust Z-Score with 1 lags
found a significant test result for column: Trust Z-Score with 2 lags
found a significant test result for column: Trust Z-Score with 2 lags
found a significant test result for column: Trust Z-Score with 2 lags
found a significant test result for column: Trust Z-Score with 2 lags
found a significant test result for column: Trust Z-Score with 4 lags
found a significant test result for column: Trust Z-Score with 4 lags
found a significant test result for column: Trust Z-Score with 4 lags
found a significant test result for column: Trust Z-Score with 4 lags
found a significant test result for column: Trust Z-Score with 5 lags
found a significant test result for column: Trust Z-Score with 5 lags
found a significant 

In [48]:
for emotion in ["Anticipation Z-Score", "Trust Z-Score", "Surprise Z-Score", "Sadness Z-Score", "Joy Z-Score", "Fear Z-Score", "Disgust Z-Score", "Anger Z-Score"]:
    print("---------", emotion, "---------")
    x = np.asarray([approvals_array, dated_emotion_df[emotion]]).T
    grangercausalitytests(x, maxlag=7)

--------- Anticipation Z-Score ---------

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.6406  , p=0.4246  , df_denom=167, df_num=1
ssr based chi2 test:   chi2=0.6521  , p=0.4194  , df=1
likelihood ratio test: chi2=0.6508  , p=0.4198  , df=1
parameter F test:         F=0.6406  , p=0.4246  , df_denom=167, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.9184  , p=0.4012  , df_denom=164, df_num=2
ssr based chi2 test:   chi2=1.8929  , p=0.3881  , df=2
likelihood ratio test: chi2=1.8823  , p=0.3902  , df=2
parameter F test:         F=0.9184  , p=0.4012  , df_denom=164, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.6225  , p=0.6014  , df_denom=161, df_num=3
ssr based chi2 test:   chi2=1.9488  , p=0.5831  , df=3
likelihood ratio test: chi2=1.9376  , p=0.5855  , df=3
parameter F test:         F=0.6225  , p=0.6014  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr bas

ssr based F test:         F=3.9198  , p=0.0217  , df_denom=164, df_num=2
ssr based chi2 test:   chi2=8.0785  , p=0.0176  , df=2
likelihood ratio test: chi2=7.8914  , p=0.0193  , df=2
parameter F test:         F=3.9198  , p=0.0217  , df_denom=164, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.7881  , p=0.0116  , df_denom=161, df_num=3
ssr based chi2 test:   chi2=11.8583 , p=0.0079  , df=3
likelihood ratio test: chi2=11.4585 , p=0.0095  , df=3
parameter F test:         F=3.7881  , p=0.0116  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=3.0737  , p=0.0180  , df_denom=158, df_num=4
ssr based chi2 test:   chi2=12.9950 , p=0.0113  , df=4
likelihood ratio test: chi2=12.5142 , p=0.0139  , df=4
parameter F test:         F=3.0737  , p=0.0180  , df_denom=158, df_num=4

Granger Causality
number of lags (no zero) 5
ssr based F test:         F=4.2047  , p=0.0013  , df_denom=155, df_num=5
ssr based chi2 test: 

In [49]:
significant_sentiments = set()
for (columnName, columnData) in sentiment_df.iteritems():
    if columnName.endswith("Z-Score"):
        find_significant_time_series(sentiment_df, columnName, significant_sentiments)

found a significant test result for column: pos_sentiment Z-Score with 6 lags
found a significant test result for column: pos_sentiment Z-Score with 6 lags
found a significant test result for column: pos_sentiment Z-Score with 6 lags
found a significant test result for column: pos_sentiment Z-Score with 6 lags
found a significant test result for column: pos_sentiment Z-Score with 7 lags
found a significant test result for column: pos_sentiment Z-Score with 7 lags
found a significant test result for column: neg_sentiment Z-Score with 6 lags
found a significant test result for column: neg_sentiment Z-Score with 6 lags
found a significant test result for column: neg_sentiment Z-Score with 6 lags
found a significant test result for column: neg_sentiment Z-Score with 6 lags
found a significant test result for column: neg_sentiment Z-Score with 7 lags
found a significant test result for column: neg_sentiment Z-Score with 7 lags
found a significant test result for column: neg_sentiment Z-Scor

In [50]:
for sentiment in ["pos_sentiment Z-Score", "neu_sentiment Z-Score", "neg_sentiment Z-Score"]:
    print("---------", sentiment, "---------")
    x = np.asarray([approvals_array, sentiment_df[sentiment]]).T
    grangercausalitytests(x, maxlag=7)

--------- pos_sentiment Z-Score ---------

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.2613  , p=0.6099  , df_denom=167, df_num=1
ssr based chi2 test:   chi2=0.2660  , p=0.6061  , df=1
likelihood ratio test: chi2=0.2657  , p=0.6062  , df=1
parameter F test:         F=0.2613  , p=0.6099  , df_denom=167, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.3130  , p=0.7317  , df_denom=164, df_num=2
ssr based chi2 test:   chi2=0.6451  , p=0.7243  , df=2
likelihood ratio test: chi2=0.6438  , p=0.7248  , df=2
parameter F test:         F=0.3130  , p=0.7317  , df_denom=164, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.6189  , p=0.6037  , df_denom=161, df_num=3
ssr based chi2 test:   chi2=1.9374  , p=0.5855  , df=3
likelihood ratio test: chi2=1.9263  , p=0.5878  , df=3
parameter F test:         F=0.6189  , p=0.6037  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr ba

In [51]:
for sentiment in significant_sentiments:
    print(sentiment)

neg_sentiment Z-Score
pos_sentiment Z-Score


In [52]:
significant_topics = set()
for (columnName, columnData) in topic_df.iteritems():
    if columnName.endswith('Z-Score'):
        find_significant_time_series(topic_df, columnName, significant_topics)

/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1768: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. T

found a significant test result for column: topic6 Z-Score with 1 lags
found a significant test result for column: topic6 Z-Score with 1 lags
found a significant test result for column: topic6 Z-Score with 1 lags
found a significant test result for column: topic6 Z-Score with 1 lags
found a significant test result for column: topic6 Z-Score with 2 lags
found a significant test result for column: topic6 Z-Score with 2 lags
found a significant test result for column: topic6 Z-Score with 2 lags
found a significant test result for column: topic6 Z-Score with 2 lags
found a significant test result for column: topic6 Z-Score with 3 lags
found a significant test result for column: topic6 Z-Score with 3 lags
found a significant test result for column: topic6 Z-Score with 3 lags
found a significant test result for column: topic6 Z-Score with 3 lags
found a significant test result for column: topic10 Z-Score with 2 lags
found a significant test result for column: topic28 Z-Score with 1 lags
foun

found a significant test result for column: topic70 Z-Score with 3 lags
found a significant test result for column: topic74 Z-Score with 6 lags
found a significant test result for column: topic79 Z-Score with 1 lags
found a significant test result for column: topic79 Z-Score with 1 lags
found a significant test result for column: topic79 Z-Score with 1 lags
found a significant test result for column: topic79 Z-Score with 1 lags
found a significant test result for column: topic79 Z-Score with 2 lags
found a significant test result for column: topic79 Z-Score with 2 lags
found a significant test result for column: topic79 Z-Score with 2 lags
found a significant test result for column: topic79 Z-Score with 2 lags
found a significant test result for column: topic79 Z-Score with 5 lags
found a significant test result for column: topic79 Z-Score with 5 lags
found a significant test result for column: topic81 Z-Score with 6 lags
found a significant test result for column: topic81 Z-Score with

found a significant test result for column: topic132 Z-Score with 7 lags
found a significant test result for column: topic132 Z-Score with 7 lags
found a significant test result for column: topic132 Z-Score with 7 lags
found a significant test result for column: topic142 Z-Score with 3 lags
found a significant test result for column: topic142 Z-Score with 3 lags
found a significant test result for column: topic142 Z-Score with 5 lags
found a significant test result for column: topic142 Z-Score with 5 lags
found a significant test result for column: topic142 Z-Score with 5 lags
found a significant test result for column: topic142 Z-Score with 5 lags
found a significant test result for column: topic142 Z-Score with 7 lags
found a significant test result for column: topic148 Z-Score with 5 lags
found a significant test result for column: topic148 Z-Score with 5 lags
found a significant test result for column: topic148 Z-Score with 6 lags
found a significant test result for column: topic14

found a significant test result for column: topic185 Z-Score with 4 lags
found a significant test result for column: topic185 Z-Score with 4 lags
found a significant test result for column: topic185 Z-Score with 4 lags
found a significant test result for column: topic185 Z-Score with 4 lags
found a significant test result for column: topic185 Z-Score with 5 lags
found a significant test result for column: topic185 Z-Score with 5 lags
found a significant test result for column: topic185 Z-Score with 5 lags
found a significant test result for column: topic185 Z-Score with 5 lags
found a significant test result for column: topic185 Z-Score with 6 lags
found a significant test result for column: topic185 Z-Score with 6 lags
found a significant test result for column: topic185 Z-Score with 6 lags
found a significant test result for column: topic185 Z-Score with 6 lags
found a significant test result for column: topic185 Z-Score with 7 lags
found a significant test result for column: topic18

...

found a significant test result for column: topic284 Z-Score with 2 lags
found a significant test result for column: topic284 Z-Score with 2 lags
found a significant test result for column: topic284 Z-Score with 2 lags
found a significant test result for column: topic284 Z-Score with 2 lags
found a significant test result for column: topic284 Z-Score with 7 lags
found a significant test result for column: topic286 Z-Score with 3 lags
found a significant test result for column: topic286 Z-Score with 3 lags
found a significant test result for column: topic286 Z-Score with 3 lags
found a significant test result for column: topic286 Z-Score with 3 lags
found a significant test result for column: topic286 Z-Score with 4 lags
found a significant test result for column: topic286 Z-Score with 4 lags
found a significant test result for column: topic286 Z-Score with 4 lags
found a significant test result for column: topic286 Z-Score with 4 lags
found a significant test result for column: topic28

found a significant test result for column: topic346 Z-Score with 1 lags
found a significant test result for column: topic346 Z-Score with 1 lags
found a significant test result for column: topic346 Z-Score with 1 lags
found a significant test result for column: topic346 Z-Score with 1 lags
found a significant test result for column: topic346 Z-Score with 2 lags
found a significant test result for column: topic347 Z-Score with 2 lags
found a significant test result for column: topic347 Z-Score with 2 lags
found a significant test result for column: topic347 Z-Score with 2 lags
found a significant test result for column: topic347 Z-Score with 2 lags
found a significant test result for column: topic349 Z-Score with 1 lags
found a significant test result for column: topic349 Z-Score with 1 lags
found a significant test result for column: topic349 Z-Score with 1 lags
found a significant test result for column: topic349 Z-Score with 1 lags
found a significant test result for column: topic34

found a significant test result for column: topic386 Z-Score with 1 lags
found a significant test result for column: topic386 Z-Score with 1 lags
found a significant test result for column: topic386 Z-Score with 1 lags
found a significant test result for column: topic386 Z-Score with 1 lags
found a significant test result for column: topic386 Z-Score with 2 lags
found a significant test result for column: topic386 Z-Score with 2 lags
found a significant test result for column: topic386 Z-Score with 2 lags
found a significant test result for column: topic386 Z-Score with 2 lags
found a significant test result for column: topic386 Z-Score with 3 lags
found a significant test result for column: topic386 Z-Score with 3 lags
found a significant test result for column: topic386 Z-Score with 3 lags
found a significant test result for column: topic386 Z-Score with 3 lags
found a significant test result for column: topic386 Z-Score with 4 lags
found a significant test result for column: topic38

found a significant test result for column: topic448 Z-Score with 7 lags
found a significant test result for column: topic452 Z-Score with 3 lags
found a significant test result for column: topic452 Z-Score with 3 lags
found a significant test result for column: topic452 Z-Score with 3 lags
found a significant test result for column: topic452 Z-Score with 3 lags
found a significant test result for column: topic452 Z-Score with 4 lags
found a significant test result for column: topic452 Z-Score with 4 lags
found a significant test result for column: topic452 Z-Score with 4 lags
found a significant test result for column: topic452 Z-Score with 4 lags
found a significant test result for column: topic452 Z-Score with 5 lags
found a significant test result for column: topic452 Z-Score with 5 lags
found a significant test result for column: topic452 Z-Score with 5 lags
found a significant test result for column: topic452 Z-Score with 5 lags
found a significant test result for column: topic45

found a significant test result for column: topic492 Z-Score with 3 lags
found a significant test result for column: topic492 Z-Score with 5 lags
found a significant test result for column: topic492 Z-Score with 5 lags
found a significant test result for column: topic492 Z-Score with 5 lags
found a significant test result for column: topic492 Z-Score with 5 lags


In [56]:
for (columnName, columnData) in topic_df.iteritems():
    if columnName.endswith('Z-Score'):
        print()
        print("---------", columnName, "---------")
        x = np.asarray([approvals_array, topic_df[columnName]]).T
        grangercausalitytests(x, maxlag=7)


--------- topic1 Z-Score ---------

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0810  , p=0.7763  , df_denom=167, df_num=1
ssr based chi2 test:   chi2=0.0824  , p=0.7740  , df=1
likelihood ratio test: chi2=0.0824  , p=0.7740  , df=1
parameter F test:         F=0.0810  , p=0.7763  , df_denom=167, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.0158  , p=0.3644  , df_denom=164, df_num=2
ssr based chi2 test:   chi2=2.0936  , p=0.3511  , df=2
likelihood ratio test: chi2=2.0807  , p=0.3533  , df=2
parameter F test:         F=1.0158  , p=0.3644  , df_denom=164, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.7632  , p=0.1563  , df_denom=161, df_num=3
ssr based chi2 test:   chi2=5.5196  , p=0.1375  , df=3
likelihood ratio test: chi2=5.4309  , p=0.1428  , df=3
parameter F test:         F=1.7632  , p=0.1563  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F 

/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1768: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. T


parameter F test:         F=2.9540  , p=0.0875  , df_denom=167, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=2.0534  , p=0.1316  , df_denom=164, df_num=2
ssr based chi2 test:   chi2=4.2319  , p=0.1205  , df=2
likelihood ratio test: chi2=4.1798  , p=0.1237  , df=2
parameter F test:         F=2.0534  , p=0.1316  , df_denom=164, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.4026  , p=0.2440  , df_denom=161, df_num=3
ssr based chi2 test:   chi2=4.3907  , p=0.2222  , df=3
likelihood ratio test: chi2=4.3343  , p=0.2276  , df=3
parameter F test:         F=1.4026  , p=0.2440  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1.5046  , p=0.2034  , df_denom=158, df_num=4
ssr based chi2 test:   chi2=6.3613  , p=0.1737  , df=4
likelihood ratio test: chi2=6.2432  , p=0.1817  , df=4
parameter F test:         F=1.5046  , p=0.2034  , df_denom=158, df_num=4

Granger Causality
n

number of lags (no zero) 3
ssr based F test:         F=0.9122  , p=0.4365  , df_denom=161, df_num=3
ssr based chi2 test:   chi2=2.8554  , p=0.4145  , df=3
likelihood ratio test: chi2=2.8314  , p=0.4183  , df=3
parameter F test:         F=0.9122  , p=0.4365  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.6644  , p=0.6176  , df_denom=158, df_num=4
ssr based chi2 test:   chi2=2.8092  , p=0.5903  , df=4
likelihood ratio test: chi2=2.7858  , p=0.5943  , df=4
parameter F test:         F=0.6644  , p=0.6176  , df_denom=158, df_num=4

Granger Causality
number of lags (no zero) 5
ssr based F test:         F=0.5961  , p=0.7030  , df_denom=155, df_num=5
ssr based chi2 test:   chi2=3.1922  , p=0.6704  , df=5
likelihood ratio test: chi2=3.1619  , p=0.6750  , df=5
parameter F test:         F=0.5961  , p=0.7030  , df_denom=155, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.6586  , p=0.6831  , df_denom=152, df_


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1507  , p=0.6984  , df_denom=167, df_num=1
ssr based chi2 test:   chi2=0.1534  , p=0.6953  , df=1
likelihood ratio test: chi2=0.1533  , p=0.6954  , df=1
parameter F test:         F=0.1507  , p=0.6984  , df_denom=167, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1236  , p=0.8838  , df_denom=164, df_num=2
ssr based chi2 test:   chi2=0.2548  , p=0.8804  , df=2
likelihood ratio test: chi2=0.2546  , p=0.8805  , df=2
parameter F test:         F=0.1236  , p=0.8838  , df_denom=164, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.5104  , p=0.6757  , df_denom=161, df_num=3
ssr based chi2 test:   chi2=1.5977  , p=0.6599  , df=3
likelihood ratio test: chi2=1.5902  , p=0.6616  , df=3
parameter F test:         F=0.5104  , p=0.6757  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.4896  , p=0.7433  

...

number of lags (no zero) 1
ssr based F test:         F=0.0090  , p=0.9246  , df_denom=167, df_num=1
ssr based chi2 test:   chi2=0.0092  , p=0.9238  , df=1
likelihood ratio test: chi2=0.0092  , p=0.9238  , df=1
parameter F test:         F=0.0090  , p=0.9246  , df_denom=167, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.1895  , p=0.3070  , df_denom=164, df_num=2
ssr based chi2 test:   chi2=2.4516  , p=0.2935  , df=2
likelihood ratio test: chi2=2.4339  , p=0.2961  , df=2
parameter F test:         F=1.1895  , p=0.3070  , df_denom=164, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.6807  , p=0.1733  , df_denom=161, df_num=3
ssr based chi2 test:   chi2=5.2612  , p=0.1536  , df=3
likelihood ratio test: chi2=5.1805  , p=0.1590  , df=3
parameter F test:         F=1.6807  , p=0.1733  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1.3223  , p=0.2640  , df_denom=158, df_

likelihood ratio test: chi2=0.9866  , p=0.9118  , df=4
parameter F test:         F=0.2341  , p=0.9189  , df_denom=158, df_num=4

Granger Causality
number of lags (no zero) 5
ssr based F test:         F=0.3146  , p=0.9036  , df_denom=155, df_num=5
ssr based chi2 test:   chi2=1.6847  , p=0.8908  , df=5
likelihood ratio test: chi2=1.6762  , p=0.8919  , df=5
parameter F test:         F=0.3146  , p=0.9036  , df_denom=155, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.3749  , p=0.8941  , df_denom=152, df_num=6
ssr based chi2 test:   chi2=2.4416  , p=0.8749  , df=6
likelihood ratio test: chi2=2.4237  , p=0.8769  , df=6
parameter F test:         F=0.3749  , p=0.8941  , df_denom=152, df_num=6

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=0.9384  , p=0.4788  , df_denom=149, df_num=7
ssr based chi2 test:   chi2=7.2300  , p=0.4053  , df=7
likelihood ratio test: chi2=7.0752  , p=0.4211  , df=7
parameter F test:         F=0.9384  , p

number of lags (no zero) 4
ssr based F test:         F=0.9785  , p=0.4210  , df_denom=158, df_num=4
ssr based chi2 test:   chi2=4.1369  , p=0.3878  , df=4
likelihood ratio test: chi2=4.0865  , p=0.3944  , df=4
parameter F test:         F=0.9785  , p=0.4210  , df_denom=158, df_num=4

Granger Causality
number of lags (no zero) 5
ssr based F test:         F=1.3032  , p=0.2654  , df_denom=155, df_num=5
ssr based chi2 test:   chi2=6.9783  , p=0.2223  , df=5
likelihood ratio test: chi2=6.8356  , p=0.2332  , df=5
parameter F test:         F=1.3032  , p=0.2654  , df_denom=155, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=1.1345  , p=0.3450  , df_denom=152, df_num=6
ssr based chi2 test:   chi2=7.3890  , p=0.2864  , df=6
likelihood ratio test: chi2=7.2284  , p=0.3002  , df=6
parameter F test:         F=1.1345  , p=0.3450  , df_denom=152, df_num=6

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=1.2067  , p=0.3025  , df_denom=149, df_

ssr based F test:         F=1.2648  , p=0.2819  , df_denom=155, df_num=5
ssr based chi2 test:   chi2=6.7728  , p=0.2381  , df=5
likelihood ratio test: chi2=6.6383  , p=0.2490  , df=5
parameter F test:         F=1.2648  , p=0.2819  , df_denom=155, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.6420  , p=0.6965  , df_denom=152, df_num=6
ssr based chi2 test:   chi2=4.1815  , p=0.6521  , df=6
likelihood ratio test: chi2=4.1294  , p=0.6592  , df=6
parameter F test:         F=0.6420  , p=0.6965  , df_denom=152, df_num=6

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=1.3223  , p=0.2435  , df_denom=149, df_num=7
ssr based chi2 test:   chi2=10.1882 , p=0.1782  , df=7
likelihood ratio test: chi2=9.8843  , p=0.1952  , df=7
parameter F test:         F=1.3223  , p=0.2435  , df_denom=149, df_num=7

--------- topic464 Z-Score ---------

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0130  , p=0.9094  , df_den

ssr based chi2 test:   chi2=4.0504  , p=0.2561  , df=3
likelihood ratio test: chi2=4.0024  , p=0.2612  , df=3
parameter F test:         F=1.2939  , p=0.2784  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.8630  , p=0.4876  , df_denom=158, df_num=4
ssr based chi2 test:   chi2=3.6485  , p=0.4557  , df=4
likelihood ratio test: chi2=3.6093  , p=0.4615  , df=4
parameter F test:         F=0.8630  , p=0.4876  , df_denom=158, df_num=4

Granger Causality
number of lags (no zero) 5
ssr based F test:         F=0.8635  , p=0.5072  , df_denom=155, df_num=5
ssr based chi2 test:   chi2=4.6237  , p=0.4635  , df=5
likelihood ratio test: chi2=4.5605  , p=0.4718  , df=5
parameter F test:         F=0.8635  , p=0.5072  , df_denom=155, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.7673  , p=0.5967  , df_denom=152, df_num=6
ssr based chi2 test:   chi2=4.9978  , p=0.5441  , df=6
likelihood ratio test: chi2=4.9236  , p

In [53]:
print(len(significant_topics))
for topic in significant_topics:
    print(topic)
    
with (open("significant_topics.csv",'w')) as file:
    writer = csv.writer(file)
    for topic in significant_topics:
        writer.writerow([topic])

124
topic252 Z-Score
topic352 Z-Score
topic301 Z-Score
topic10 Z-Score
topic366 Z-Score
topic79 Z-Score
topic418 Z-Score
topic349 Z-Score
topic457 Z-Score
topic109 Z-Score
topic97 Z-Score
topic369 Z-Score
topic462 Z-Score
topic150 Z-Score
topic358 Z-Score
topic105 Z-Score
topic196 Z-Score
topic231 Z-Score
topic292 Z-Score
topic115 Z-Score
topic186 Z-Score
topic473 Z-Score
topic167 Z-Score
topic482 Z-Score
topic403 Z-Score
topic434 Z-Score
topic492 Z-Score
topic148 Z-Score
topic221 Z-Score
topic203 Z-Score
topic322 Z-Score
topic346 Z-Score
topic376 Z-Score
topic443 Z-Score
topic262 Z-Score
topic185 Z-Score
topic333 Z-Score
topic276 Z-Score
topic157 Z-Score
topic49 Z-Score
topic437 Z-Score
topic461 Z-Score
topic50 Z-Score
topic198 Z-Score
topic113 Z-Score
topic319 Z-Score
topic28 Z-Score
topic476 Z-Score
topic321 Z-Score
topic261 Z-Score
topic114 Z-Score
topic240 Z-Score
topic297 Z-Score
topic303 Z-Score
topic263 Z-Score
topic74 Z-Score
topic165 Z-Score
topic357 Z-Score
topic265 Z-Score


# Example of Granger Causality Test output 

In [54]:
x = np.asarray([approvals_array, dated_emotion_df["Trust Z-Score"]]).T
grangercausalitytests(x, maxlag=7)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=5.4970  , p=0.0202  , df_denom=167, df_num=1
ssr based chi2 test:   chi2=5.5957  , p=0.0180  , df=1
likelihood ratio test: chi2=5.5056  , p=0.0190  , df=1
parameter F test:         F=5.4970  , p=0.0202  , df_denom=167, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.4721  , p=0.0334  , df_denom=164, df_num=2
ssr based chi2 test:   chi2=7.1558  , p=0.0279  , df=2
likelihood ratio test: chi2=7.0085  , p=0.0301  , df=2
parameter F test:         F=3.4721  , p=0.0334  , df_denom=164, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.2669  , p=0.0828  , df_denom=161, df_num=3
ssr based chi2 test:   chi2=7.0963  , p=0.0689  , df=3
likelihood ratio test: chi2=6.9506  , p=0.0735  , df=3
parameter F test:         F=2.2669  , p=0.0828  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.8883  , p=0.0242  

{1: ({'lrtest': (5.505609732254527, 0.018955570283706202, 1),
   'params_ftest': (5.496983958196578, 0.020225482168538346, 167.0, 1.0),
   'ssr_chi2test': (5.595732173014359, 0.018004284631948128, 1),
   'ssr_ftest': (5.496983958196458, 0.020225482168539945, 167.0, 1)},
   array([[0., 1., 0.]])]),
 2: ({'lrtest': (7.008467158653616, 0.030069810639428818, 2),
   'params_ftest': (3.4720538653599906, 0.03335465188161074, 164.0, 2.0),
   'ssr_chi2test': (7.155818332265652, 0.027934042682227347, 2),
   'ssr_ftest': (3.4720538653596655, 0.03335465188162115, 164.0, 2)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'lrtest': (6.950560617461974, 0.07349037987026029, 3),
   'params_ftest': (2.2668879158290665, 0.08277238345468022, 161.0, 3.0),
   'ssr_chi2test': (7.096344779986445, 0.06888951812583004, 3),
   'ssr_ftest': (2.2668879158290034, 0.08277238345468578, 161.0, 3)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0.